In [54]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import TextRequestsWrapper
from langchain.memory import ConversationBufferMemory

os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

#llm=OpenAI(temperature=0)
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

In [55]:
def search_SMILES(drugname):
    # pubchem API to get SMILES 
    requests = TextRequestsWrapper()
    response = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{drugname}/property/CanonicalSMILES/json")

    return response

In [56]:
py_executor = create_python_agent(
    llm = ChatOpenAI(model_name='gpt-3.5-turbo'),
    tool=PythonREPLTool(),
    verbose=True
)

In [57]:
tools = [
    Tool(
        name = "Search SMILES",
        func = lambda s: search_SMILES(s),
        description = "useful for when you need to search the SMILES by drugname."
    ),
    Tool(
        name = "Calcuation by Python",
        func = py_executor.run,
        description = "useful for when you need to exeute python to do the calculation"
    )
]

In [60]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [61]:
agent.run("Search the SMILES of Abiraterone")
agent.run("Search the SMILES of Abacavir")
agent.run("Execte python code to calculate similarity score between Asprin and Abacavir in RDkit until there is no bug issue")



> Entering new AgentExecutor chain...
{
    "action": "Search SMILES",
    "action_input": "Abiraterone"
}
Observation: {
  "PropertyTable": {
    "Properties": [
      {
        "CID": 132971,
        "CanonicalSMILES": "CC12CCC(CC1=CCC3C2CCC4(C3CC=C4C5=CN=CC=C5)C)O"
      }
    ]
  }
}

Thought:{
    "action": "Final Answer",
    "action_input": "The SMILES for Abiraterone is CC12CCC(CC1=CCC3C2CCC4(C3CC=C4C5=CN=CC=C5)C)O"
}

> Finished chain.


> Entering new AgentExecutor chain...
{
    "action": "Search SMILES",
    "action_input": "Abacavir"
}
Observation: {
  "PropertyTable": {
    "Properties": [
      {
        "CID": 441300,
        "CanonicalSMILES": "C1CC1NC2=C3C(=NC(=N2)N)N(C=N3)C4CC(C=C4)CO"
      }
    ]
  }
}

Thought:{
    "action": "Final Answer",
    "action_input": "The SMILES for Abacavir is C1CC1NC2=C3C(=NC(=N2)N)N(C=N3)C4CC(C=C4)CO"
}

> Finished chain.


> Entering new AgentExecutor chain...
{
    "action": "Calcuation by Python",
    "action_input": "import nu

OutputParserException: Could not parse LLM output: `We have imported numpy and rdkit libraries and generated molecular fingerprints for two molecules. We have also calculated the similarity between the two fingerprints. 
Action: None`